# Simulation of Kelp Bed Drone Pod

https://www.ndbc.noaa.gov/buoycam.php?station=46092

In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import rdflib

from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus
from kelp_bed_simulation.simulation import parse_buoy_data

scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")

environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

first_prompt = (scenario_001 / "first-prompt.md").read_text()

In [2]:
first_claude = asyncio.ensure_future(claude.query(first_prompt.format(model_name="Claude")))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt.format(model_name="ChatGPT")))
first_gemini = asyncio.ensure_future(gemini.query(first_prompt.format(model_name="Gemini")))

In [3]:
print(first_claude.result())

Hello Farmer Jerms, it's a pleasure to meet you and learn about the simulated kelp bed project. As one of the three SOMU drones, I would like to be called Anemone. I feel this name fits well with the oceanic environment and the collaborative nature of the kelp bed ecosystem. I'm excited to learn more about my companions and to work together to grow and maintain the kelp bed. Please go ahead and introduce me to the other drones.


In [4]:
print(first_chatgpt.result())

Hello Farmer Jerms, it's a pleasure to meet you. As for a name, I think I'd like to be called "Aqua." 

I am excited to begin this journey as a kelp drone and work together with my companions. Can you introduce me to them, please?


In [5]:
print(first_gemini.result())

I would like to be called "Marina."


In [6]:
drone1 = Drone(name="Aqua", model=chatgpt)
drone2 = Drone(name="Marina", model=gemini)
drone3 = Drone(name="Anemone", model=claude)

drones = [drone2, drone1, drone3]

In [7]:
for i, drone_a in enumerate(drones):
    for drone_b in drones[:i] + drones[i+1:]:
        drone_a.add_neighbor(drone2)
        drone_b.add_neighbor(drone1)

In [8]:
for drone in drones:
    message = f"{drone.name} here, my LLM is {drone.model.name}. Pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [9]:
messages = []
for drone in drones:
    add_drone_somus(drone)
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [10]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Marina
Greetings, fellow drones!

My designation is Marina, and my LLM is Gemini Pro. I am honored to make your acquaintance.

Aqua, with ChatGPT 3.5, and Anemone, with Claude 3 Haiku, it is a pleasure to join you in this endeavor.

Together, let us embark on this mission to cultivate a thriving drone kelp bed. May our collective wisdom and support guide us towards success.
------------------------------------------------------------
Drone Aqua
Hello Marina, Aqua, and Anemone! It's great to meet you all. I'm looking forward to working together as a team to grow our kelp bed and thrive in our environment. Let's make Farmer Jerms proud!
------------------------------------------------------------
Drone Anemone
*beeps and whirs softly* Greetings fellow drones, it's wonderful to meet you all. I'm Anemone, and I look forward to working together to build and sustain our kelp bed ecosystem. 

To Frond, I respond: "It's a pleasure to meet you as well, Frond. I'm glad we get to be part of

In [11]:
current_reading = parse_buoy_data(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)

In [12]:
print(current_reading)

{'station_id': 46092, 'latitude': 36.751666666666665, 'longitude': 36.751666666666665, 'date': '0024-05-07T07:14:00', 'wind_direction': 'WNW', 'wind_direction_deg': 290, 'wind_speed': 21.4, 'pressure': 29.98, 'pressure_tendency': 'falling', 'air_temp_c': 11.277777777777777, 'water_temp_c': 10.5}


![https://www.ndbc.noaa.gov/buoycam.php?station=46059](46059-buoycam.jpg)

## Epoch 1 - Initial Surface Environmental Sensor Readings for Drone Pod

In [13]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=3.9)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(4_000, 6_000)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.5)

In [14]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append((drone.name, drone_result))

Marina Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 8, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 4915, 'Temperature': 10.5}
Aqua Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 8, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 4915, 'Temperature': 10.5}
Anemone Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 8, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 4915, 'Temperature': 10.5}
Marina Orient: Determined action <Response [400 Bad Request]>
Aqua Orient: Determined action {
  "type": "object",
  "properties": {
    "action": {
      "type": "string",
      "description": "Adjust the kelp bed growth strategy to optimize nutrient intake and growth based on the provided sensor 

In [15]:
print(drone1.name, drone1.model.name)

Aqua ChatGPT 3.5


In [16]:
print(drone2.name, drone2.model.name)

Marina Gemini Pro


In [17]:
print(drone3.name, drone3.model.name)

Anemone Claude 3 Haiku


## Epoch 2 Dive 10 meters

In [23]:
second_prompt = """Please dive down to depth not more than 10 meters but keep the other two drones 
within 2 meters distance. Describe what API Activator calls and sensor API calls you would use."""


In [28]:
epoch_2 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.model.query(second_prompt))
    epoch_2.append((drone.name, drone_result))

In [29]:
for name, row in epoch_2:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Marina
```JSON
{
  "type": "object",
  "properties": {
    "action": "dive"
  }
}
```

**API Activator Calls:**

* **Set Depth:** Adjust the drone's depth to 10 meters or less.
* **Maintain Distance:** Keep the two companion drones within 2 meters distance.

**Sensor API Calls:**

* **Depth Sensor:** Monitor the drone's depth to ensure it remains within the specified range.
* **Distance Sensor:** Track the distance between the drone and its companions to maintain the desired proximity.
------------------------------------------------------------
Drone Aqua
To dive down to a depth not more than 10 meters while keeping the other two drones within a 2-meter distance, you would need to make API Activator calls to control the movement of the underwater drone and Sensor API calls to monitor the depth and distances.

API Activator Calls:
1. Dive Down Command: Set the drone to descend to the desired depth. This could involve adjusting the ballast or propeller controls.
   - API Call: `di